In [ ]:
from joblib import load
from matplotlib import pyplot as plt
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.neural_network import MLPClassifier
import gurobipy as gp
from gurobipy import GRB

In [ ]:
# import my functions
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')
from ml2grb.sklearn2grb import Pipe2Gurobi

from ml2grb.activations2grb import Identity
from ml2grb.extra.obbt import obbt
from ml2grb.extra.morerelu import reluOBBT
from ml2grb.extra.morerelu import ReLUmin

In [ ]:
# Load the trained network and the examples
pipe = load('MNIST_50_50.joblib')
X = load('MNIST_first100.joblib')

In [ ]:
#Choose an example
exampleno = 13
example = X[exampleno,:]
ex_prob = pipe.predict_proba(example.reshape(1,-1))
output_shape = ex_prob.shape[1]

In [ ]:
pixels = example.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pipe.predict(example.reshape(1, -1))

In [ ]:
sortedidx = np.argsort(ex_prob)[0]

In [ ]:
m = gp.Model()
epsilon = 7
lb = np.maximum(example - epsilon, 0)
ub = np.minimum(example + epsilon, 1)

x = m.addMVar(example.shape, lb=lb, ub=ub, name='X')
absdiff = m.addMVar(example.shape, lb=0, ub=1, name='dplus')
output = m.addMVar(output_shape, lb=-gp.GRB.INFINITY, name='y')

In [ ]:
m.addConstr(absdiff >= x - example)
m.addConstr(absdiff >= - x + example)
m.addConstr(absdiff.sum() <= epsilon)
pipe2grb = Pipe2Gurobi(pipe, m)
# For this example we should model softmax in the last layer using identity
pipe2grb.steps[-1].actdict['softmax'] = Identity()
pipe2grb.predict(x, output)
m.setObjective(output[sortedidx[-2]] - output[sortedidx[-1]], gp.GRB.MAXIMIZE)
m.update()

In [ ]:
obbt(pipe2grb.steps[-1], activation=reluOBBT('both'))

In [ ]:
m.Params.OutputFlag = 1
m.Params.MIRCuts = 2
m.Params.TimeLimit = 250
m.Params.BestObjStop = 0.0
m.Params.BestBdStop = 0.0
m.optimize()

In [ ]:
pixels = x.X.reshape((28, 28))
plt.imshow(pixels, cmap='gray')
plt.show()

In [ ]:
pipe.predict(x.X.reshape(1, -1))

In [ ]:
pipe.predict_proba(x.X.reshape(1, -1)).round(3)

In [ ]:
pipe2grb.steps[-1]._layers[2].actvar.UB

pipe2grbght © 2022 Gurobi Optimization, LLC